In [1]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from sbi_scraping import get_own_stock_df
from minkabu_scraping import append_dividend
from tse_stock_list_downloader import append_market_and_industries

import config

import pandas as pd
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
driver = webdriver.Chrome(
    service=ChromeService(ChromeDriverManager().install())
)  # Mac用の記述
driver.implicitly_wait(10)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8.81M/8.81M [00:00<00:00, 25.3MB/s]


In [3]:
# 証券会社のwebサイトから保有株情報を抽出する
df_own_stock = get_own_stock_df(driver)

In [4]:
# 配当金額を抽出する
df_own_stock = append_dividend(driver, df_own_stock)

1434 start
1434 finish
1518 start
1518 finish
1775 start
1775 finish
1820 start
1820 finish
1835 start
1835 finish
1925 start
1925 finish
1928 start
1928 finish
1951 start
1951 finish
2124 start
2124 finish
2127 start
2127 finish
2169 start
2169 finish
2317 start
2317 finish
2353 start
2353 finish
2395 start
2395 finish
2914 start
2914 finish
2927 start
2927 finish
3003 start
3003 finish
3101 start
3101 finish
3156 start
3156 finish
3179 start
3179 finish
3186 start
3186 finish
3248 start
3248 finish
3409 start
3409 finish
3632 start
3632 finish
3817 start
3817 finish
3847 start
3847 finish
4008 start
4008 finish
4042 start
4042 finish
4044 start
4044 finish
4064 start
4064 finish
4182 start
4182 finish
4361 start
4361 finish
4385 start
4385 skipped
4452 start
4452 finish
4502 start
4502 finish
4641 start
4641 finish
5101 start
5101 finish
5184 start
5184 finish
5201 start
5201 finish
5406 start
5406 skipped
5713 start
5713 finish
5803 start
5803 finish
6073 start
6073 finish
6087 star

In [3]:
# これ以降の動作だけテストしたい時は下記コードを利用する
df_own_stock = pd.read_csv("test_dataframe.csv")
df_own_stock["コード"] = df_own_stock["コード"].astype(object)
df_own_stock["市場"] = df_own_stock["市場"].astype(object)
df_own_stock["業種"] = df_own_stock["業種"].astype(object)

In [4]:
df_own_stock = append_market_and_industries(df_own_stock)

In [5]:
# 2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# 認証情報設定
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "keys/credentials.json", scope
)

In [6]:
# OAuth2の資格情報を使用してGoogle APIにログイン
gc = gspread.authorize(credentials)

In [11]:
# 共有設定したスプレッドシートを開く
spreadsheet = gc.open_by_key(config.SPREADSHEET_KEY)

In [12]:
# シート名を指定してシートを開く
worksheet = spreadsheet.worksheet("②高配当株ポートフォリオ")